<a href="https://colab.research.google.com/github/Frank-Howd/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/module2-make-features/LS_DS_112_Make_Features_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 1, Sprint 1, Module 2*

---

# Learning Objectives

- Student should be able to understand the purpose of feature engineering
- Student should be able to work with strings in pandas
- Student should be able to work with dates and times in pandas
- Student should be able to modify or create columns of a dataframe using the `.apply()` function


Helpful Links:
- [Minimally Sufficient Pandas](https://medium.com/dunder-data/minimally-sufficient-pandas-a8e67f2a2428)
- [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)
- Python Data Science Handbook
  - [Chapter 3.10](https://jakevdp.github.io/PythonDataScienceHandbook/03.10-working-with-strings.html), Vectorized String Operations
  - [Chapter 3.11](https://jakevdp.github.io/PythonDataScienceHandbook/03.11-working-with-time-series.html), Working with Time Series
- [Lambda Learning Method for DS - By Ryan Herr](https://docs.google.com/document/d/1ubOw9B3Hfip27hF2ZFnW3a3z9xAgrUDRReOEo-FHCVs/edit?usp=sharing)

# Let's continue to explore feature engineering with the Ames Housing data.  Import the dataset and name it "house".

###1) Import the Ames Housing Data here: https://raw.githubusercontent.com/ryanleeallred/datasets/master/Ames%20Housing%20Data/train.csv

In [30]:
#Code here

import numpy as np
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/Ames%20Housing%20Data/train.csv')

###2) Print the first five rows of the house dataset.

In [31]:
#Code here

df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


###3) Create a new variable called Large_Lot that takes on the value 1 when LotArea is gerater than or equal to 10,000 and the value 0 when LotArea is less than 10,000.

In [32]:
#Code here

df.loc[df['LotArea'] >= 10000, 'Large_Lot'] = 1
df.loc[df['LotArea'] < 10000, 'Large_Lot'] = 0

pd.crosstab(df['Large_Lot'], columns='Counts')

col_0,Counts
Large_Lot,
0.0,837
1.0,623


###4) Create a new variable called Total_Bathrooms that contains the total number of full and half bathrooms in the house.

In [33]:
#Code here

df['Total_Bathrooms'] = df['BsmtFullBath'] + df['BsmtHalfBath'] + df['FullBath'] 
+ df['HalfBath']

df['Total_Bathrooms'].value_counts()

2    762
3    341
1    329
4     27
6      1
Name: Total_Bathrooms, dtype: int64

###5) Create a new variable called Summer_Sale that equals 1 if the sale month was in June, July or August ( MoSold = 6, 7 or 8) and 0 if the sale month was in September - May.

In [38]:
#code here

def monthSold(moSoldInt):
    if moSoldInt >= 6 and moSoldInt <= 8:
        return 1
    elif moSoldInt > 8 or moSoldInt < 6:
        return 0

df['Summer_Sale'] = [monthSold(x) for x in df['MoSold']]

df['Summer_Sale'].value_counts()


0    851
1    609
Name: Summer_Sale, dtype: int64

#Now we'll revisit the LendingClub data.

Remember to import the data by first unzipping the file:

In [43]:
!wget https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip

--2020-12-01 21:12:05--  https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip
Resolving resources.lendingclub.com (resources.lendingclub.com)... 52.12.152.84, 35.162.47.238, 54.191.199.187
Connecting to resources.lendingclub.com (resources.lendingclub.com)|52.12.152.84|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘LoanStats_2018Q4.csv.zip.2’

LoanStats_2018Q4.cs     [              <=>   ]  22.37M  6.14MB/s    in 4.5s    

2020-12-01 21:12:10 (5.02 MB/s) - ‘LoanStats_2018Q4.csv.zip.2’ saved [23452842]



In [44]:
!unzip LoanStats_2018Q4.csv.zip


Archive:  LoanStats_2018Q4.csv.zip
replace LoanStats_2018Q4.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: LoanStats_2018Q4.csv    


In [45]:
# Read in the CSV
import pandas as pd
df = pd.read_csv('LoanStats_2018Q4.csv', skiprows=1, skipfooter=2, engine='python')

print(df.shape)
df.head()

(128412, 144)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,6000,6000,6000.0,36 months,16.14%,211.36,C,C4,Facility Maintenance Tech,3 years,RENT,40000.0,Source Verified,Dec-2018,Fully Paid,n,NaN,NaN,other,Other,217xx,MD,3.75,0,Aug-2008,0,NaN,NaN,9,0,5580,10.4%,15,w,0.00,0.00,6425.400634,6425.40,...,0.0,0,0,55383,6785,48500,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,25000,25000,25000.0,36 months,16.91%,890.20,C,C5,Captain,3 years,RENT,67000.0,Source Verified,Dec-2018,Current,n,NaN,NaN,debt_consolidation,Debt consolidation,671xx,KS,13.18,0,Aug-1996,3,46.0,NaN,16,0,17053,29.3%,20,w,10499.56,10499.56,20439.370000,20439.37,...,10.0,0,0,66400,23438,42400,8200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,40000,40000,40000.0,60 months,11.31%,875.90,B,B3,NaN,< 1 year,MORTGAGE,136000.0,Not Verified,Dec-2018,Current,n,NaN,NaN,debt_consolidation,Debt consolidation,480xx,MI,15.41,0,Apr-2006,0,72.0,NaN,12,0,34283,81%,22,w,29112.11,29112.11,18356.200000,18356.20,...,75.0,0,0,179159,62143,25000,42409,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,CVD19SKIP,INCOMECURT,COMPLETE,2.0,0.0,Apr-2020,May-2020,Mar-2020,2.0,2.0,GRACE_CURRENT,613.57,32887.60,875.90,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,35000,35000,35000.0,60 months,15.02%,833.02,C,C3,elementary school teacher,10+ years,MORTGAGE,106095.0,Source Verified,Dec-2018,Current,n,NaN,NaN,debt_consolidation,Debt consolidation,017xx,MA,15.38,1,Mar-1998,0,2.0,NaN,9,0,19518,83.1%,14,w,24536.67,24536.67,19115.650000,19115.65,...,100.0,0,0,499675,72376,21000,72510,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,5000,5000,5000.0,36 months,11.31%,164.43,B,B3,Shipping,2 years,RENT,40000.0,Source Verified,Dec-2018,Current,n,NaN,NaN,debt_consolidation,Debt consolidation,284xx,NC,9.66,0,Oct-2007,0,NaN,NaN,8,0,1070,17.5%,27,w,2497.47,2497.47,3285.470000,3285.47,...,0.0,0,0,125853,144979,3600,119753,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,CVD19SKIP,INCOMECURT,COMPLETE,2.0,0.0,Apr-2020,Jun-2020,May-2020,2.0,0.0,ACTIVE,58.80,3121.41,164.43,N,NaN,NaN,NaN,NaN,NaN,NaN


#Approaching a problem two different ways.

In the guided project, we learned how to find the earliest credit year using the built-in pandas date-time format.  However, there is often more than one way to come up with the same solution.  







In the following questions we will work through the steps to create a new variables called Earliest_Credit_Year and issue_year where we use the .split() function remove the month report back just the year as a float data type.  Then we'll use those variables to calculate the length of credit history in years.  

If we do everything correctly, we should calculate *about* the same length of longest credit history that we did in class working with the date-time format.  The answers won't exactly match because we are making different assumptions about when in each month and year the loans were taken out, but it will give you a flavor for different ways of approaching the same problem.

###6) Create a simple test case where credit = 'Jun-1979'.  

Use the .split('-') function to separate the month and year parts of the data and print the year.

Hint: Name the results of the .split('-') function "fields".  fields[0] will return the month part of the date and fields[1] will return the year part of the date.

In [48]:
#code here

credit = 'Jun-1979'

fields = credit.split('-')
print(fields[0])
fields[1]


Jun


'1979'

###7) Use your answer in 6) to write a function called credit_yr that takes in the contents of a cell formatted 'month-year' and returns the year.

Run your function using the simple test case in 6).

In [58]:
#Code here

def credit_yr(monthYr):
    '''Takes to contents of a cell formatted 'month-year' and returns 
    the year'''
    
    x = monthYr.split('-')
    return int(x[1])

print(credit_yr(credit))

1979


###8) Use the .apply() function to apply the function to every cell in the earliest_cr_line variable.  Create a new variable called Earliest_Credit_Year that contains the results of the function.

In [59]:
#code here

df['Earliest_Credit_Year'] = df['earliest_cr_line'].apply(credit_yr)


###9) Print the top 5 rows of the Earliest_Credit_Year and earliest_cr_line variables to make sure the variables were created correctly.

In [66]:
#code here

print(df[['Earliest_Credit_Year', 'earliest_cr_line']].head())
#df['Earliest_Credit_Year'][:5]

   Earliest_Credit_Year earliest_cr_line
0                  2008         Aug-2008
1                  1996         Aug-1996
2                  2006         Apr-2006
3                  1998         Mar-1998
4                  2007         Oct-2007


### 10) Use your function to create a new variable called Issue_Year from the issue_d variable.

In [64]:
#code here

df['Issue_Year'] = df['issue_d'].apply(credit_yr)
df['Issue_Year'].value_counts()

2018    128412
Name: Issue_Year, dtype: int64

###11) Caluculate length of credit history in years and in days.

Hint: If you are getting an error message, make sure to check the data type of the years output by your functions.

In [81]:
#Code here

#Years
df['Credit_Length_Years'] = df['Issue_Year'] - df['Earliest_Credit_Year']

#Days
df['Credit_Length_Days'] = (df['Issue_Year'] - df['Earliest_Credit_Year'])*365.25

print(df['Credit_Length_Years'].head())
print(df['Credit_Length_Days'].head())

# df['issue_d'] = pd.to_datetime(df['issue_d'], infer_datetime_format= True)
# df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'], infer_datetime_format=True)

# print(df[['issue_d', 'earliest_cr_line']].head())

# df['credit_length_days'] = (df['issue_d'] - df['earliest_cr_line']).dt.days
# df['credit_length_years'] = df['credit_length_days'] / 365.25


0    10
1    22
2    12
3    20
4    11
Name: Credit_Length_Years, dtype: int64
0    3652.50
1    8035.50
2    4383.00
3    7305.00
4    4017.75
Name: Credit_Length_Days, dtype: float64
68


24837.0

###12) Calculate the maximum credit length and compare it to we got in the guided project.  

Note - it won't be *exactly* the same because both methods are working in different ways, but they will give you a "pretty close" answer.

In [84]:
#code here

print(df['Credit_Length_Years'].max()) #Guided Project - 68.91444216290212
df['Credit_Length_Days'].max() #25171

68


24837.0

#Portfolio Project Assignment

Start looking for interesting datasets!  

Pretty much every major US governmental agency (CDC, Bureau of Labor Statistics, The Census, etc.) maintains a treasure trove of data.

There's also:

Datasets from the University of Florida Department of Statistics: http://users.stat.ufl.edu/~winner/datasets.html

Pew Research: https://www.pewresearch.org/

General Social Survey: https://gss.norc.org/

Ancestry.com Free data collections: https://www.ancestry.com/search/categories/freeindexacom/?clickref=1101lbhdWxys&key=Uhttps%3A%2F%2Fwww.ancestry.com%2Fsearch%2Fcategories%2Ffreeindexacom%2F%3Fclickref%3D1101lbhdWxys&o_lid=01011l4pVw&o_sch=Affiliate+External&o_xid=01011l4pVw

Popular baby names by year since the founding of the SSA: https://www.babynamewizard.com/ 


https://data.world/

https://www.kaggle.com/datasets

https://catalog.data.gov/dataset

https://datasetsearch.research.google.com/

https://archive.ics.uci.edu/ml/index.php

https://github.com/plotly/datasets

https://lionbridge.ai/datasets/20-best-image-datasets-for-computer-vision/

https://www.visualdata.io/

https://pathmind.com/wiki/open-datasets

https://blog.playment.io/top-open-image-dataset/

https://blog.cambridgespark.com/

https://www.analyticsvidhya.com/blog/2018/03/

https://data-flair.training/blogs/machine-learning-datasets/





